In [ ]:
import pandas as pd
import torch
from torch import nn 


In [ ]:
folder = '/media/username/DATA_LINUX1/Datasets/Odeuropa/benchmarks_and_corpora/benchmarks/EN/webanno'

In [ ]:
model_architecture = "emanjavacas/MacBERTh"

#model_architecture = "bert-base-uncased"
#model_architecture = "roberta-base"

In [ ]:
random_value = 42

In [ ]:


def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


In [ ]:
# import required module
import os
 
 
# iterate over files in
# that directory
count=0

print(os.walk(folder))

dataframes = []
for subfolder in sorted(os.listdir(folder)):
        subfolder_path = subfolder
        tables = []
        subfolder_path = os.path.join(folder, subfolder_path)
        for file in os.scandir(subfolder_path):
            if file.is_file():
                annotation_path = os.path.join(subfolder_path, file)

                table = pd.read_table(annotation_path,comment='#', error_bad_lines=False, engine="python", header=None, quoting = 3, quotechar=None)
                table = table.rename(columns={0: "token_id", 1: "char_range", 2: "token", 3: "ref_type"})
                
                if not 'ref_type' in table:
                    continue
                #table = table[~(table['ref_type'] == '_') & ~table['ref_type'].isna()]
                table['filename'] = annotation_path.split('/')[-2]
                tables.append(table)
                print(annotation_path)
                
        dataframes.append(pd.concat(tables, ignore_index=True, axis=0))


In [ ]:
docs_folder = '/media/username/DATA_LINUX1/Datasets/Odeuropa/benchmarks_and_corpora/benchmarks/EN/source/source'

In [ ]:
from nltk import tokenize
from tqdm import tqdm
tqdm.pandas()

text_content = {}
for file in sorted(os.listdir(docs_folder)):
    with open(os.path.join(docs_folder, file)) as f:
        lines = f.read()

        lines = tokenize.sent_tokenize(lines)
        lines =[line.replace('\n', ' ') for line in lines]
        lines =[line.replace('\t', ' ') for line in lines]
        text_content[file] = lines

In [ ]:
refs = pd.concat(dataframes)

In [ ]:
refs['sentence_id'] = refs['token_id'].apply(lambda x : x.split('-')[0])

In [ ]:
refs = refs.drop_duplicates(subset=['filename', 'sentence_id', 'token_id'])

In [ ]:
refs = refs[~refs['ref_type'].isna()]

In [ ]:
refs

In [ ]:
def get_ref_type(ref_type):
    if '[' in ref_type:
        return str(ref_type).split('[')[0]
    else:
        return ref_type

In [ ]:
refs['ref_type'] = refs['ref_type'].apply(lambda x : 'O' if x == '_' else '1' )

In [ ]:
refs = refs.reset_index(drop='true')

In [ ]:
sentences = refs.groupby(['filename', 'sentence_id'], as_index = False).agg({'token':  ' '.join})
encodings = refs.groupby(['filename', 'sentence_id'], as_index = False).agg({'ref_type': ' '.join}).rename(columns = {'ref_type':'labels'})


In [ ]:
def mark_beginning_and_intermediate_token(label):
    tokens = label.split()
    
    for i in range(len(tokens)):
        if tokens[i] != 'O' :
            tokens[i] = 'B-'+ tokens[i]
            
    for i in range(len(tokens)-1):
        if tokens[i] != 'O' and tokens[i+1] != 'O':
            if tokens[i+1].replace("B-", '') in tokens[i]:
                tokens[i+1] = tokens[i+1].replace("B-", "I-")
            
    
    return ' '.join(tokens)

In [ ]:
encodings['labels'] = encodings['labels'].apply(mark_beginning_and_intermediate_token)

In [ ]:
df = pd.merge(sentences, encodings, on=["filename", "sentence_id"])
df = df.rename(columns={'token': 'text'})

In [ ]:
def contains_ref(labels):
    labels = labels.split()
    for label in labels:
        if label != "O":
            return True
    return False

In [ ]:
df['contains_ref'] = df['labels'].apply(contains_ref)

In [ ]:
df['contains_ref'].value_counts()

In [ ]:
df['text'].to_list()[3]

In [ ]:
def spellcheck(sentence):
    spell = SpellChecker()
    sentence = sentence.split()
    
    corrected_sentence = [spell.correction(word) if spell.correction(word) is not None else word for word in sentence ]
    return ' '.join(corrected_sentence) if corrected_sentence is not None else sentence

In [ ]:


def normalize(comment, lowercase=True, remove_stopwords=True):
    if lowercase:
        comment = comment.lower()
    comment = spacy_model(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not word.is_stop and not word.is_punct and word.pos_ in ['NOUN', 'ADJ', 'ADV', 'VERB']:
                lemmatized.append(lemma)
    return " ".join(lemmatized)

In [ ]:
from sklearn.model_selection import train_test_split

import spacy
spacy_model = spacy.load("en_core_web_md")


from spellchecker import SpellChecker


tqdm.pandas()

In [ ]:
# # removing the new line characters
# with open('sound_sentences_17_25_words_300.csv') as f:
#     lines = [line.rstrip() for line in f]
 
# lines = [line for line in lines if line !='']

In [ ]:
generated_sentences = pd.DataFrame({'text':lines})
generated_sentences['spellchecked_text'] = generated_sentences['text'].progress_apply(spellcheck)
generated_sentences['normalized_text'] = generated_sentences['spellchecked_text'].progress_apply(normalize)

In [ ]:
generated_sentences['contains_ref'] = True

In [ ]:
# utterance_words = [str(x).split() for  x in train_data['normalized_text'].to_list()]

In [ ]:
# ut = []

# for l in utterance_words:
#     ut+=l

In [ ]:
# ct = 0
# for l in ut:
    
#     if sensory_tag_token(l, 'Visual', dict_lexicon) != 0:
#         ct+=1
        

In [ ]:
# ct/len(ut)

In [ ]:
# ut =list( dict.fromkeys(ut) ) 

In [ ]:


#df['spellchecked_text'] = df['text'].progress_apply(spellcheck)
#df['normalized_text'] = df['spellchecked_text'].progress_apply(normalize)

df = pd.read_csv('cleansed_odeuropa_sentences.csv')
df = pd.concat([df, generated_sentences], axis=0).reset_index(drop=True)

train_data, test_data = train_test_split(df, test_size=0.1, random_state=random_value)
#train_data, train_mlp_data = train_test_split(train_data, test_size=0.75, random_state=42)

In [ ]:
train_data

In [ ]:
train_data['contains_ref'].value_counts()

In [ ]:
mapping = {False: 0, True:1}

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(model_architecture, do_lower_case=True)


In [ ]:
train_sentences = train_data['text'].to_list()
train_labels = [mapping[x] for x in train_data['contains_ref'].to_list()]

max_len = 0

# For every sentence...
for sent in train_sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)


In [ ]:
def sensory_tag_token(token, sensoriality, dict_lexicon):
    if token in dict_lexicon:
        return dict_lexicon[token][sensoriality]
    else:
        synonyms = wordnet.synsets(token)
        candidates = list(set(chain.from_iterable([word.lemma_names() for word in synonyms])))
        
        for candidate in candidates:
            if candidate in dict_lexicon:
                return dict_lexicon[candidate][sensoriality]
        
        return 0

In [ ]:
def sensory_tag_sentence(sentence, nlp):
    s = 0

    doc = nlp(sentence)
    
    sensory_dict = {'AUD': 0, 'GUS': 0, 'HAP': 0, 'INT': 0, 'OLF': 0, 'VIS': 0, 'ARM': 0, 
                    'LEG': 0, 'TORSO':0, 'HEAD':0, 'MOUTH':0
                   }
    
    for token in doc:
        if token.pos_ in ['NOUN', 'ADJ', 'ADV', 'VERB']:
            

            
            sensory_dict['AUD'] += sensory_tag_token(token.lemma_, 'Auditory', dict_lexicon)
            sensory_dict['GUS'] += sensory_tag_token(token.lemma_, 'Gustatory', dict_lexicon)
            sensory_dict['HAP'] += sensory_tag_token(token.lemma_, 'Haptic', dict_lexicon)
            sensory_dict['INT'] += sensory_tag_token(token.lemma_, 'Interoceptive', dict_lexicon)
            sensory_dict['OLF'] += sensory_tag_token(token.lemma_, 'Olfactory', dict_lexicon)
            sensory_dict['VIS'] += sensory_tag_token(token.lemma_, 'Visual', dict_lexicon)
            
            sensory_dict['ARM'] += sensory_tag_token(token.lemma_, 'Hand_arm', dict_lexicon)
            sensory_dict['LEG'] += sensory_tag_token(token.lemma_, 'Foot_leg', dict_lexicon)
            sensory_dict['TORSO'] += sensory_tag_token(token.lemma_, 'Torso', dict_lexicon)
            sensory_dict['HEAD'] += sensory_tag_token(token.lemma_, 'Head', dict_lexicon)
            sensory_dict['MOUTH'] += sensory_tag_token(token.lemma_, 'Mouth', dict_lexicon)
            
            s+=1
    
#     for key in sensory_dict.keys():
#         if s > 0:
#             sensory_dict[key] /= s
    return sensory_dict

In [ ]:
test_sentences = test_data['text'].to_list()
test_labels = [mapping[x] for x in test_data['contains_ref'].to_list()]

max_len = 0

# For every sentence...
for sent in test_sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
from itertools import chain
from nltk.corpus import wordnet


def normalize(comment, lowercase=True, remove_stopwords=True):
    if lowercase:
        comment = comment.lower()
    comment = spacy_model(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            #if not word.is_stop and not word.is_punct and word.pos_ in ['NOUN', 'ADJ', 'ADV', 'VERB']:
            lemmatized.append(lemma)
    return " ".join(lemmatized)

In [ ]:
lexicon_model = pd.read_csv('/media/username/DATA_LINUX1/Datasets/Lancaster/Lancaster_sensorimotor_norms_for_39707_words.csv')
lexicon = lexicon_model[['Word', 'Auditory.mean', 'Gustatory.mean', 'Haptic.mean', 'Interoceptive.mean', 'Olfactory.mean', 'Visual.mean', 'Foot_leg.mean', 'Hand_arm.mean', 'Head.mean', 'Mouth.mean', 'Torso.mean']]
lexicon = lexicon.rename(columns={"Auditory.mean": "Auditory", "Gustatory.mean": "Gustatory", "Haptic.mean": "Haptic", "Interoceptive.mean": "Interoceptive", "Olfactory.mean": "Olfactory", "Visual.mean": "Visual", 'Torso.mean': 'Torso', 'Mouth.mean': 'Mouth', 'Head.mean': 'Head', 'Foot_leg.mean': 'Foot_leg', 'Hand_arm.mean': 'Hand_arm'})

In [ ]:
lexicon['Word'] = lexicon['Word'].apply(lambda x: x.lower())
lexicon = lexicon.set_index(['Word'])
dict_lexicon = lexicon.to_dict('index')

In [ ]:
texts = df['normalized_text'].to_list()
split_captions= [str(caption).split(' ') for caption in texts]

In [ ]:
import fasttext
fasttext_path = './wiki.en.bin'
fasttext_model = fasttext.load_model(fasttext_path)
fasttext_model.get_nearest_neighbors('dog', k=10)

In [ ]:
wordnet.synsets('dog')

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
train_input_ids = []
train_attention_masks = []
train_sensorimotor_embeddings = []

# For every sentence...
for sent in train_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    train_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    train_attention_masks.append(encoded_dict['attention_mask'])
    
    train_sensorimotor_embeddings.append(list(sensory_tag_sentence(sent, spacy_model).values()))

# Convert the lists into tensors.
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_sensorimotor_embeddings = torch.stack([torch.tensor(sent) for sent in train_sensorimotor_embeddings])
train_labels = torch.tensor(train_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])


# Tokenize all of the sentences and map the tokens to thier word IDs.
test_input_ids = []
test_attention_masks = []
test_sensorimotor_embeddings = []

# For every sentence...
for sent in test_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])
    test_sensorimotor_embeddings.append(list(sensory_tag_sentence(sent, spacy_model).values()))


# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
test_sensorimotor_embeddings = torch.stack([torch.tensor(sent) for sent in test_sensorimotor_embeddings])
test_labels = torch.tensor(test_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', test_sentences[0])
print('Token IDs:', test_input_ids[0])


In [ ]:
torch.stack([torch.tensor(sent) for sent in train_sensorimotor_embeddings]).shape

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_sensorimotor_embeddings, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_sensorimotor_embeddings, test_labels)


# # Create a 90-10 train-validation split.

# # Calculate the number of samples to include in each set.
# train_size = int(0.9 * len(dataset))
# val_size = len(dataset) - train_size

# # Divide the dataset by randomly selecting samples.
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))


In [ ]:
train_dataset

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


In [ ]:
from transformers.utils import ModelOutput
from transformers import  BertConfig, BertPreTrainedModel, BertModel
from typing import List, Optional, Tuple, Union

class SequenceClassifierOutput(ModelOutput):
    """
    Base class for outputs of sentence classification models.

    Args:
        loss (`torch.FloatTensor` of shape `(1,)`, *optional*, returned when `labels` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (`torch.FloatTensor` of shape `(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (`tuple(torch.FloatTensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `torch.FloatTensor` (one for the output of the embeddings, if the model has an embedding layer, +
            one for the output of each layer) of shape `(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the optional initial embedding outputs.
        attentions (`tuple(torch.FloatTensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `torch.FloatTensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
    """

    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [ ]:
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss, MSELoss

linear_size = 32

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size + 11, config.num_labels)
        
#         self.classifier = nn.Sequential(
#             nn.Linear(config.hidden_size + 11, linear_size),
#             nn.ReLU(),
#             nn.Linear(linear_size, linear_size),
#             nn.ReLU(),            
#             nn.Linear(linear_size, 2)
#         )

        # Initialize weights and apply final processing
        self.post_init()

#     @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
#     @add_code_sample_docstrings(
#         checkpoint=_CHECKPOINT_FOR_SEQUENCE_CLASSIFICATION,
#         output_type=SequenceClassifierOutput,
#         config_class=_CONFIG_FOR_DOC,
#         expected_output=_SEQ_CLASS_EXPECTED_OUTPUT,
#         expected_loss=_SEQ_CLASS_EXPECTED_LOSS,
#     )
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        sensorimotor_embeddings: Optional[torch.Tensor] = None
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(torch.cat((pooled_output, sensorimotor_embeddings), 1))

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
# model = BertForSequenceClassification.from_pretrained(
#     "emanjavacas/MacBERTh", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 11, # The number of output labels--2 for binary classification.
#                     # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = False, # Whether the model returns all hidden-states.
# )


model = BertForSequenceClassification.from_pretrained(
    model_architecture, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()



In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


In [ ]:
model.classifier

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.optim import Adam
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 30

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [ ]:
class BertPooler(nn.Module):
    def __init__(self, config):
        super(BertPooler, self).__init__()
        self.dense = nn.Linear(config['hidden_size'], config['hidden_size'])
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

In [ ]:
def get_pooled_outputs(bert, input_ids, attention_masks, token_type_ids=None,):
    bert_pooler = BertPooler({'hidden_size':768})
    bert_pooler.cuda()
#     embedding_output = bert.bert.embeddings(input_ids, token_type_ids)
#     print(embedding_output.shape)
#     print(attention_masks.shape)
#     encoded_layers = bert.bert.encoder(embedding_output,
#                                   attention_masks.T)
    
#     sequence_output = encoded_layers[-1]
    sequence_output = model(input_ids.to('cuda'), attention_masks.to('cuda'))[1][12]
    pooled_output = bert_pooler(sequence_output)
    encoded_layers = None
    
    return encoded_layers, pooled_output


In [ ]:
import random
import numpy as np
import time
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = random_value

# bert_pooler = BertPooler({'hidden_size':768})
# bert_pooler.cuda()

device = 'cuda'

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

best_accuracy = 0

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    train_hidden_states = []
    test_hidden_states = []
    
    t_train_hidden_states = []
    t_test_hidden_states = []
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    
    train_true = []
    train_preds = []

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_sensorimotor_emb = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        
        # #         sequence_output = model(b_input_ids.to('cuda'), b_input_mask.to('cuda'))[1][12]
#         pooled_output = model.bert.pooler(model(b_input_ids.to('cuda'), b_input_mask.to('cuda'))[1][12]).to('cpu')
#         train_hidden_states.append(pooled_output)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        output = model(b_input_ids, 
                               token_type_ids=None, 
                               attention_mask=b_input_mask,
                               sensorimotor_embeddings=b_sensorimotor_emb,
                               labels=b_labels)

# #         sequence_output = model(b_input_ids.to('cuda'), b_input_mask.to('cuda'))[1][12]
#         pooled_output = model.bert.pooler(model(b_input_ids.to('cuda'), b_input_mask.to('cuda'))[1][12]).to('cpu')
#         train_hidden_states.append(pooled_output)


        loss = output['loss']
        logits = output['logits']
        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        train_true.append(label_ids)
        train_preds.append(logits)
            

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    true = []
    preds = []

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_sensorimotor_emb = batch[2].to(device)
        b_labels = batch[3].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            output = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   sensorimotor_embeddings=b_sensorimotor_emb,
                                   labels=b_labels)
            
#             sequence_output = model(b_input_ids.to('cuda'), b_input_mask.to('cuda'))[1][12].to('cpu')
#             pooled_output = model.bert.pooler(model(b_input_ids.to('cuda'), b_input_mask.to('cuda'))[1][12]).to('cpu')
#             test_hidden_states.append(pooled_output)

            loss = output['loss']
            logits = output['logits']
                    # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            true.append(label_ids)
            preds.append(logits)
                        
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        


        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))


        
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    
    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        torch.save(model.state_dict(), 'best-model-parameters.pt') # official recommended
        
        best_preds = preds
        best_true = true
        
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


In [ ]:
model

In [ ]:
# train_hidden_states = torch.cat(train_hidden_states, dim=0)
# test_hidden_states = torch.cat(test_hidden_states, dim=0)

# # t_train_hidden_states = torch.cat(t_train_hidden_states, dim=0)
# # t_test_hidden_states = torch.cat(t_test_hidden_states, dim=0)

In [ ]:
# train_hidden_states.shape

In [ ]:
from sklearn.metrics import confusion_matrix

import seaborn as sn
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
logits = preds
preds = [np.argmax(i, axis=1).flatten() for i in preds]
preds = np.concatenate(preds)
true = np.concatenate(true)

In [ ]:
logits = preds
best_preds = [np.argmax(i, axis=1).flatten() for i in best_preds]
best_preds = np.concatenate(best_preds)
best_true = np.concatenate(best_true)

In [ ]:
cf_matrix = confusion_matrix(true, preds)

classes = list(mapping.keys())

def_classes = []

for indice in np.unique(true):
    def_classes.append(classes[indice])

# classes = ['Circumstances', 'Effect', 'Location', 'Perceiver', 'Quality', 'Smell\\_Source', 'Time']

df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in def_classes],
                     columns = [i for i in def_classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('{}_bs_{}_factorized_classes.png'.format(model_architecture.replace('/', '_'), batch_size))

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
true

In [ ]:
precision = precision_score(best_true, best_preds, average="macro", pos_label=1)
recall = recall_score(best_true, best_preds, average="macro", pos_label=1)

print('Precision: ',precision)
print('Recall: ',recall)

In [ ]:
output[2][0].shape

In [ ]:
# train_preds = [np.argmax(i, axis=1).flatten() for i in train_preds]
# train_preds = np.concatenate(train_preds)
# train_true = np.concatenate(train_true)

# precision = precision_score(train_true, train_preds, average="macro", pos_label=1)
# recall = recall_score(train_true, train_preds, average="macro", pos_label=1)

# print('Precision: ',precision)
# print('Recall: ',recall)

In [ ]:
from pytorch_pretrained_bert import BertModel
class ExtraBertMultiClassifier(nn.Module):
    def __init__(self, bert_model_path=model_architecture, labels_count=2, hidden_dim=768, mlp_dim=100, extras_dim=11, dropout=0.1):
        super().__init__()

        self.config = {
            'bert_model_path': model_architecture,
            'labels_count': 2,
            'hidden_dim': 768,
            'mlp_dim': 100,
            'extras_dim': 11,
            'dropout': 0.1,
        }

        self.bert = model
        self.dropout = nn.Dropout(self.config['dropout'])
        self.mlp = nn.Sequential(
            nn.Linear(self.config['hidden_dim'] + self.config['extras_dim'], self.config['mlp_dim']),
            nn.ReLU(),
            nn.Linear(self.config['mlp_dim'], self.config['mlp_dim']),
            nn.ReLU(),            
            nn.Linear(self.config['mlp_dim'], self.config['labels_count'])
        )
    
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()

    def forward(self, tokens, masks, extras):
#         _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        _, pooled_output = get_pooled_outputs(self.bert, tokens, masks)
        dropout_output = self.dropout(pooled_output)
        
        concat_output = torch.cat((pooled_output, extras), dim=1)
        mlp_output = self.mlp(concat_output)
        # proba = self.sigmoid(mlp_output)
        proba = self.softmax(mlp_output)

        return proba


In [ ]:
train_data['sensorimotor_representation'] = train_data.progress_apply(lambda x: sensory_tag_sentence(str(x.normalized_text), spacy_model), axis=1)
test_data['sensorimotor_representation'] = test_data.progress_apply(lambda x: sensory_tag_sentence(str(x.normalized_text), spacy_model), axis=1)

In [ ]:
train_df = pd.concat([train_data.drop(['sensorimotor_representation'], axis=1), train_data['sensorimotor_representation'].apply(pd.Series)], axis=1)
test_df = pd.concat([test_data.drop(['sensorimotor_representation'], axis=1), test_data['sensorimotor_representation'].apply(pd.Series)], axis=1)

In [ ]:
# train_df['bert_pred'] = train_preds
# test_df['bert_pred'] = preds

In [ ]:
features = [ 'GUS', 'HAP', 'AUD', 'OLF', 'VIS', 'INT', 'MOUTH', 'HEAD', 'TORSO', 'ARM', 'LEG']
train_mlp_tensor = torch.tensor(train_df[features].values).float()
test_mlp_tensor = torch.tensor(test_df[features].values).float()

In [ ]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_mlp_tensor, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_mlp_tensor, test_labels)


In [ ]:
batch_size = 16
train_mlp_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_mlp_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


In [ ]:
# logits = preds
# preds = [np.argmax(i, axis=1).flatten() for i in preds]
# preds = np.concatenate(preds)
# true = np.concatenate(true)
# precision = precision_score(true, preds, average="macro", pos_label=1)
# recall = recall_score(true, preds, average="macro", pos_label=1)

# print('Precision: ',precision)
# print('Recall: ',recall)

In [ ]:
model_architecture

In [ ]:
def train(classifier, optimizer, train_dataloader):
    epochs = 10
    loss_func = nn.BCELoss()

    for epoch_num in range(epochs):
        model.train()
        train_loss = 0

        print(f'Epoch: {epoch_num + 1}/{epochs}')

        # for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        for step_num, batch_data in enumerate(tqdm(train_dataloader, desc="Iteration")):
            # Full features
            
            token_ids, masks, extras, gold_labels = tuple(t.to(device) for t in batch_data)
            probas = classifier(token_ids.to('cuda'), masks.to('cuda'), extras.to('cuda'))

            batch_loss = loss_func(torch.argmax(probas, axis=1).type(torch.FloatTensor).to('cuda'), gold_labels.type(torch.FloatTensor).to('cuda'))
            batch_loss.requires_grad = True
            train_loss += batch_loss.item()
            
            classifier.zero_grad()
            batch_loss.backward()
            optimizer.step()

            # clear_output(wait=True)

        print(f'\r{epoch_num} loss: {train_loss / (step_num + 1)}')

        print(str(torch.cuda.memory_allocated(device) / 1000000) + 'M')

    return classifier


In [ ]:
def evaluation(classifier, data_loader):

    # Validation
    model.eval()

    output_ids = []
    outputs = None
    golden_labels_list = None

    with torch.no_grad():
        for step_num, batch_item in enumerate(tqdm(data_loader, desc="Iteration")):
            
            # Full features
            token_ids, masks, extras, golden_labels = tuple(t.to(device) for t in batch_item)
            probas = classifier(token_ids.to('cuda'), masks.to('cuda'), extras.to('cuda'))

            numpy_logits = probas.cpu().detach().numpy()

            if outputs is None:
                outputs = numpy_logits
            else:
                outputs = np.vstack((outputs, numpy_logits))
                
                
            golden_labels_np = golden_labels.cpu().detach().numpy()
            
            if golden_labels_list is None:
                golden_labels_list = golden_labels_np
            else:
                golden_labels_list = np.concatenate((golden_labels_list, golden_labels_np), axis=None)

            output_ids.append(None)

    print(f'Evaluation completed for {len(outputs)} items')

    return output_ids, outputs, golden_labels_list

In [ ]:
mlp = ExtraBertMultiClassifier()
mlp.cuda()

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                )

In [ ]:
# mlp = train(mlp, optimizer, train_mlp_dataloader)


In [ ]:
# output_ids, output, golden_labels_list = evaluation(mlp, validation_mlp_dataloader)

In [ ]:
# mlp_preds = torch.argmax(torch.tensor(output), axis=1).numpy()

In [ ]:
# mlp_true = golden_labels_list

In [ ]:
# precision = precision_score(mlp_true, mlp_preds, average="macro", pos_label=1)
# # recall = recall_score(mlp_true, mlp_preds, average="macro", pos_label=1)

# print('Precision: ',precision)
# print('Recall: ',recall)

In [ ]:
# mlp_preds

In [ ]:
# model.bert.embeddings

In [ ]:
train_df['class'] = train_df['contains_ref'].apply(lambda x: 1 if x else 0)
test_df['class'] = test_df['contains_ref'].apply(lambda x: 1 if x else 0)

In [ ]:

features = [ 'GUS', 'HAP', 'AUD', 'OLF', 'VIS', 'INT', 'MOUTH', 'HEAD', 'TORSO', 'ARM', 'LEG']

# features = 'sensorimotor_representation'
columns = 'fasttext_representation'
#columns = features

# train_X = train_data[columns]
# train_X = np.asarray([x for x in train_X])
# train_X = np.concatenate((train_X, train_data[features].to_numpy()), axis=1)

train_X = train_df[features].to_numpy()
train_y = train_df['class'].to_numpy().astype('int')

# test_X = test_data[columns]
# test_X = np.asarray([x for x in test_X])
# test_X = np.concatenate((test_X, test_data[features].to_numpy()), axis=1)

test_X = test_df[features].to_numpy()
test_y = test_df['class'].to_numpy().astype('int')

In [ ]:
from sklearn.metrics import f1_score


from sklearn.linear_model import LogisticRegression
clf =LogisticRegression()

clf.fit(train_X, train_y)
lr_preds = clf.predict(test_X).astype('int')
precision = precision_score(test_y, lr_preds,  average="macro")
recall = recall_score(test_y, lr_preds,  average="macro")
f1 = f1_score(test_y, lr_preds, average="macro")
print(precision)
print(recall)
print(f1)

In [ ]:
test_data['bert_preds'] = best_preds
test_data['lr_preds'] = lr_preds


In [ ]:
test_data['bert_and_lr_preds'] =  test_data['bert_preds'] + test_data['lr_preds'] >=1
test_data['bert_and_lr_preds'] = test_data['bert_and_lr_preds'].apply(lambda x: 1 if x >0 else 0)

In [ ]:
precision = precision_score(test_y, test_data['bert_preds'], average="macro", pos_label=1)
recall = recall_score(test_y, test_data['bert_preds'],  average="macro", pos_label=1)
f1 = f1_score(test_y, test_data['bert_preds'],  average="macro", pos_label=1)
print(precision)
print(recall)
print(f1)

In [ ]:
l_precision = precision_score(test_y, test_data['bert_and_lr_preds'], average="macro", pos_label=1)
l_recall = recall_score(test_y, test_data['bert_and_lr_preds'],  average="macro", pos_label=1)
l_f1 = f1_score(test_y, test_data['bert_and_lr_preds'],  average="macro", pos_label=1)

print(l_precision)
print(l_recall)
print(l_f1)

In [ ]:
print('Precision: ',l_precision - precision)
print('Recall: ',l_recall - recall)
print('F1:', l_f1 - f1)

In [ ]:
test_data